In [1]:
from prepare_language_folder import prepare_language_folder
from preprocessing.data_collection.multipleye_data_collection import MultipleyeDataCollection
from pathlib import Path

In [2]:
data_collection_name = 'MultiplEYE_SQ_CH_Zurich_1_2025'

If necessary, prepare the data folder by unzipping the downloaded files. Works only for MultiplEYE and MeRID data collections so far. Also, there might be some manual steps necessary.

In [3]:
prepare_language_folder(data_collection_name)

No zipped data found for test_sessions. Proceeding with existing data.
No zipped data found for .DS_Store. Proceeding with existing data.
No zipped data found for 006_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for 010_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for 009_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for 007_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for pilot_sessions. Proceeding with existing data.
No zipped data found for 008_SQ_CH_1_ET1. Proceeding with existing data.
No zipped data found for 011_SQ_CH_1_ET1. Proceeding with existing data.


In [4]:
this_repo = Path().resolve()
data_folder_path = this_repo / "data" / data_collection_name

multipleye_sq = MultipleyeDataCollection.create_from_data_folder(data_folder_path)
multipleye_sq

Folder test_sessions does not match the regex pattern \d\d\d_SQ_CH_1_ET\d. Not considered as session.
Folder pilot_sessions does not match the regex pattern \d\d\d_SQ_CH_1_ET\d. Not considered as session.


Parsing participant data : 011_SQ_CH_1_ET1: 100%|██████████| 6/6 [00:00<00:00, 685.40it/s]


data_collection_name	MultiplEYE_SQ_CH_Zurich_1_2025
num_sessions	6
num_pilots	0

In [5]:
multipleye_sq.create_sanity_check_report(overwrite=False)

Creating sanity check report for session 011_SQ_CH_1_ET1: 100%|██████████| 6/6 [02:51<00:00, 28.61s/it]


In [6]:
multipleye_sq.preprocess_eye_tracking_data()

Experiment(screen=Screen(width_px=1308, height_px=1001, width_cm=54.3, height_cm=30.2, distance_cm=60, origin=None), eyetracker=EyeTracker(sampling_rate=1000.0, left=None, right=None, model=None, version=None, vendor=None, mount=None))
shape: (3_457_901, 11)
┌──────────┬────────┬────────────┬──────────┬───┬────────────┬────────────┬────────────┬───────────┐
│ time     ┆ pupil  ┆ trial      ┆ activity ┆ … ┆ session    ┆ pixel      ┆ position   ┆ velocity  │
│ ---      ┆ ---    ┆ ---        ┆ ---      ┆   ┆ ---        ┆ ---        ┆ ---        ┆ ---       │
│ i64      ┆ f64    ┆ str        ┆ str      ┆   ┆ str        ┆ list[f64]  ┆ list[f64]  ┆ list[f64] │
╞══════════╪════════╪════════════╪══════════╪═══╪════════════╪════════════╪════════════╪═══════════╡
│ 2629468  ┆ 1085.0 ┆ PRACTICE_t ┆ reading  ┆ … ┆ 006_SQ_CH_ ┆ [46.3,     ┆ [-22.78821 ┆ [-1.62550 │
│          ┆        ┆ rial_1     ┆          ┆   ┆ 1_ET1      ┆ 126.2]     ┆ 4, -10.645 ┆ 7, -0.635 │
│          ┆        ┆            ┆